In [2]:
#!pip install pytorch-tabnet wget
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn import tree
from xgboost import plot_tree
import pandas as pd
import numpy as np
import os
import wget
from pathlib import Path
import shutil
import gzip
import joblib
import pydot
from matplotlib import pyplot as plt
from xgboost import XGBClassifier

random_state=42
np.random.seed(random_state)

In [5]:
df = pd.read_csv('../../../data/DNN-EdgeIIoT-dataset.csv', low_memory=False)
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df)

categorical_columns = []
for col in df.columns[df.dtypes == object]:
    if col != "Attack_type":
        categorical_columns.append(col)

featuresFromStart = [ col for col in df.columns if col not in ["Attack_label"]+["Attack_type"]]
#print("-----Features from the start-----")
#print(featuresFromStart)
#print("-----Categorial features-----")
#print(categorical_columns)
#Display information about dataframe
def displayInformationDataFrame(df_cop):
    summary_df = pd.DataFrame(columns=['Data Type', 'Column Name', 'Unique Values'])
    # Iterate through the columns of the original dataframe
    for col in df_cop.columns:
        # Get the data type of the column
        dtype = df_cop[col].dtype
        # Get the column name
        col_name = col
        # Get the unique values of the column
        unique_values = df_cop[col].unique()
        # Append a new row to the summary dataframe
        summary_df = summary_df.append({'Data Type': dtype, 'Column Name': col_name, 'Unique Values': unique_values}, ignore_index=True)
    # display the summary_df
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    #return display(summary_df)
    
displayInformationDataFrame(df)

colunas_one_hot = {}
for coluna in categorical_columns:
    codes, uniques = pd.factorize(df[coluna].unique())
    colunas_one_hot[coluna] = {"uniques": uniques, "codes":codes}
    df[coluna] = df[coluna].replace(colunas_one_hot[coluna]["uniques"], colunas_one_hot[coluna]["codes"])
    print(coluna)
df = pd.get_dummies(data=df, columns=categorical_columns)
displayInformationDataFrame(df)

df = shuffle(df)
n_total = len(df)

features = [ col for col in df.columns if col not in ["Attack_label"]+["Attack_type"]] 

le = LabelEncoder()
le.fit(df["Attack_type"].values)

train_val_indices, test_indices = train_test_split(range(n_total), test_size=0.2, random_state=random_state)
train_indices, valid_indices = train_test_split(train_val_indices, test_size=0.25, random_state=random_state) # 0.25 x 0.8 = 0.2

X_train = df[features].values[train_val_indices]
y_train = df["Attack_type"].values[train_val_indices]
y_train = le.transform(y_train)

X_valid = df[features].values[valid_indices]
y_valid = df["Attack_type"].values[valid_indices]
y_valid = le.transform(y_valid)

X_test = df[features].values[test_indices]
y_test = df["Attack_type"].values[test_indices]
y_test = le.transform(y_test)

standScaler = StandardScaler()
model_norm = standScaler.fit(X_train)

X_train = model_norm.transform(X_train)
X_test = model_norm.transform(X_test)
X_valid = model_norm.transform(X_valid)

n_estimators = 100 if not os.getenv("CI", False) else 20

clf_xgb = XGBClassifier(max_depth=8,
    learning_rate=0.1,
    n_estimators=n_estimators,
    verbosity=0,
    silent=None,
    #objective="multi:softmax",
    objective="multi:softprob",
    booster='gbtree',
    n_jobs=1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=random_state,
    seed=None,
    num_class= (le.classes_).size)
    #num_class= 2)


http.request.method
http.referer
http.request.version
dns.qry.name.len
mqtt.conack.flags
mqtt.protoname
mqtt.topic


In [7]:

clf_xgb = clf_xgb.load_model("XGBClassifier.json")

XGBoostError: [10:15:42] /Users/travis/build/dmlc/xgboost/include/xgboost/json.h:65: Invalid cast, from Null to Array
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000146de8a60 dmlc::LogMessageFatal::~LogMessageFatal() + 112
  [bt] (1) 2   libxgboost.dylib                    0x0000000146e2eae3 xgboost::JsonArray const* xgboost::Cast<xgboost::JsonArray const, xgboost::Value>(xgboost::Value*) + 739
  [bt] (2) 3   libxgboost.dylib                    0x0000000146f0546f xgboost::RegTree::LoadModel(xgboost::Json const&) + 1519
  [bt] (3) 4   libxgboost.dylib                    0x0000000146e8aff7 xgboost::gbm::GBTreeModel::LoadModel(xgboost::Json const&) + 775
  [bt] (4) 5   libxgboost.dylib                    0x0000000146e7b0e3 xgboost::gbm::GBTree::LoadModel(xgboost::Json const&) + 419
  [bt] (5) 6   libxgboost.dylib                    0x0000000146e8d39d xgboost::LearnerIO::LoadModel(xgboost::Json const&) + 2173
  [bt] (6) 7   libxgboost.dylib                    0x00

In [3]:
clf.feature_importances_

AttributeError: 'NoneType' object has no attribute 'feature_importances_'

In [21]:
def sortSecond(val):
	return val[1]
values = clf.feature_importances_
original_labels_list = le.classes_
importances = [(features[i], np.round(values[i],4)) for i in range(len(features))]
importances.sort(reverse=True, key=sortSecond)
importances

[('mqtt.topic_0', 0.4946),
 ('udp.stream', 0.1192),
 ('tcp.ack', 0.1044),
 ('icmp.seq_le', 0.0728),
 ('http.referer_1', 0.0496),
 ('http.request.method_1', 0.045),
 ('tcp.ack_raw', 0.0319),
 ('tcp.seq', 0.0277),
 ('tcp.connection.rst', 0.0262),
 ('tcp.len', 0.0082),
 ('http.content_length', 0.0032),
 ('http.request.version_0', 0.0028),
 ('http.request.version_2', 0.0028),
 ('tcp.connection.syn', 0.0025),
 ('tcp.flags', 0.0024),
 ('http.request.version_3', 0.0019),
 ('tcp.connection.fin', 0.0014),
 ('tcp.connection.synack', 0.001),
 ('tcp.checksum', 0.0009),
 ('http.request.method_0', 0.0007),
 ('tcp.flags.ack', 0.0004),
 ('http.request.version_1', 0.0004),
 ('arp.opcode', 0.0),
 ('arp.hw.size', 0.0),
 ('icmp.checksum', 0.0),
 ('icmp.unused', 0.0),
 ('http.response', 0.0),
 ('http.tls_port', 0.0),
 ('udp.time_delta', 0.0),
 ('dns.qry.name', 0.0),
 ('dns.qry.qu', 0.0),
 ('dns.qry.type', 0.0),
 ('dns.retransmission', 0.0),
 ('dns.retransmit_request', 0.0),
 ('dns.retransmit_request_in', 0